In [1]:
import numpy as np
import igl
import meshplot as mp
import math
import scipy.sparse as sp
from numpy import matlib as mb
from math import sqrt

# Load Data

In [51]:
v, f = igl.read_triangle_mesh('../data/hand.off')

kdmax, kdmin, kmax, kmin = igl.principal_curvature(v,f)  
kdmax /= np.linalg.norm(kdmax)
kdmin /= np.linalg.norm(kdmin)
mp.plot(v, f, c=kmax)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

In [52]:
def getAdjacentFaces(v,f):
    VF, NI = igl.vertex_triangle_adjacency(f, v.shape[0])
    VFi = []
    for i in range(NI.shape[0] - 1):
        VFii = []
        jj = NI[i + 1] - NI[i]
        for j in range(jj):
            VFii.append(VF[NI[i] + j])
        VFi.append(VFii)
    return VFi

# Find Regular Triangle

In [53]:
def sgn(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    return 0

def mark_regular(f,kdmax,kdmin):
    singular = []
    regular = []
    color = []
    for indfi, fi in enumerate(f):
        sign_min = 1
        sign_max = 1
        for i in range(3):
            for j in range(i+1,3):
                sign_min *= np.dot(kdmin[fi[i]],kdmin[fi[j]])
                sign_max *= np.dot(kdmax[fi[i]],kdmax[fi[j]])
        if sign_min > 0 and sign_max > 0:
            regular.append(indfi)
            color.append([1,1,1])
        else:
            singular.append(indfi)
            color.append([1,0,0])
    return np.array(regular), np.array(singular), np.array(color)

regular, singular,color = mark_regular(f,kdmax,kdmin)
print(singular)

[    4     6    23    27    89   126   130   158   588   592   593   595
   929  1028  1152  1172  1173  1249  1622  2159  2180  2261  2442  2445
  2447  2736  2737  2739  3056  3078  3084  3086  3525  3755  3781  4046
  4141  4156  4557  4562  4696  5057  5213  5501  5603  6330  6829  7133
  7305  7862  9017  9144  9147  9730  9961 10001 10627 10987 11404 11578
 11591 11617 11645 11736 11886 12826 12939 13247 14327 14985 15010 15613
 15638 15883 16425 16428 16631 16641 17403 17842 17877 18352 19980 25368
 25369 25371 25784 25787 26221 26736 26739 26896 26899 27089 27091 27120
 27121 27123 27184 27193 27195 27204 27293 27677 27679 27800 27912 27913
 27915 28008 28011 28189 28248 28372 28373 28375 28464 28465 28467 28568]


# Compute Extremalities

In [54]:
def extremalities(v, f, k, kd):
    EX = np.zeros((v.shape[0],1))
    A = igl.doublearea(v,f)
    G = igl.grad(v, f)
    grad_k = G * k
    neighbour = getAdjacentFaces(v,f) 
    for indvi, vi in enumerate(v):
        area = 0
        ex = 0
        for vfi in neighbour[indvi]:
            area += A[vfi] # compute area star
        for indvfi, vfi in enumerate(neighbour[indvi]):
            g = np.array([grad_k[vfi], grad_k[f.shape[0] + vfi], grad_k[2 * f.shape[0] + vfi]])
            ex += A[vfi] * np.dot(g,kd[indvi])
        ex /= area
        EX[indvi] = ex
    return EX

EX_max = extremalities(v, f, kmax, kdmax)
EX_min = extremalities(v, f, kmin, kdmin)
mp.plot(v, f, c=EX_min)
mp.plot(v, f, c=EX_max)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

# Smooth Extremalities

In [55]:
def smooth_extremalities(v, f, kd, ex):
    A = igl.adjacency_list(f)
    L = igl.cotmatrix(v,f)
    M = igl.massmatrix(v,f)
    L = M.T * L
    LEx = np.zeros((v.shape[0],1))
    for indvi, vi in enumerate(v):
        lp = 0
        for vn in A[indvi]:
            sign = sgn(np.dot(kd[indvi],kd[vn]))
            lp += L[indvi,vn] * (sign * ex[vn] - ex[indvi])
        LEx[indvi] = lp
    ex += LEx
    return ex

EX_max = smooth_extremalities(v, f, kdmax, EX_max)
EX_min = smooth_extremalities(v, f, kdmin, EX_min)
mp.plot(v, f, c=EX_min)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

# Extract Feature Line

In [71]:
def extract_feature_line(v, f, kmax, kmin, KD, EX, regular, singular, sign):
    # for regular faces
    fr = np.array([f[index] for index in regular])
    TT, TTi = igl.triangle_triangle_adjacency(fr) # id of the adjacent triangle, id of the shared edge
    edges = [[],[]]
    marked_edges = []
#     G_main = igl.grad(v,fr)
#     GEX = G_main * EX
    count = 0
#     print(GEX.shape)
#     print(fr.shape)
    # for regular triangles
    for indfri, fri in enumerate(fr):
        ex = np.array([EX[index] for index in fri]) # each vertex's extremalities on the current triangle
        kd = np.array([KD[index] for index in fri]) # each vertex's curvature direction on the current triangle
        # flip signs
        change = False
        for i in range(1,3):
            if np.dot(kd[0],kd[i]) >= 0:
                kd[i] = -kd[i]
                change = True
            else:
                ex[i] = -ex[i]
        if change:
            kd[0] = -kd[0]
#         if ((ex.array() == 0.0).any())
# 			continue; // skip zeros
# 		if (ex.maxCoeff() <= 0.0 || ex.minCoeff() >= 0.0)
# 			continue; // no zero points
        kd_sum = kd.sum(axis=0)
        vv = np.array([v[index] for index in fri]) # the vertex on this triangle
        G = igl.grad(vv, np.array([[0,1,2]]))
        gex = G * ex # trangle based gradient
#         print(gex)
#         ind = indfri
#         gex = np.array([GEX[ind][0],GEX[fr.shape[0] + ind][0],GEX[2 * fr.shape[0] + ind][0]])
        # check equation (5)
        if sign * np.dot(np.array([kd_sum]),gex) >= 0: 
#         if sign * np.dot(np.array([gex[0][0],gex[1][0],gex[2][0]]), np.array([kd_sum[0],kd_sum[1],kd_sum[2]])) >= 0: 
#         if sign * np.dot(gex, np.array([kd_sum[0],kd_sum[1],kd_sum[2]])) >= 0: 
            continue
            
        # check equation (6)
        kmax_ = np.array([kmax[index] for index in fri])
        kmin_ = np.array([kmin[index] for index in fri])
        if sign * (abs(kmax_.sum(axis=0)) - abs(kmin_.sum(axis=0))) <= 0:
            continue
        count = 0
        for i in range(3):
            j = (i + 1) % 3
            if ex[i] * ex[j] < 0: # has sign change: should have a zero point in the middle
                # add mid point
                a = abs(ex[i])
                b = abs(ex[j])
                mid = (b * v[fri[i]] + a * v[fri[j]]) / (a + b)
                edges[count].append(mid)
                # mark edge i, j
                marked_edges.append([fri[i],fri[j]])
                count = (count + 1)%2
                
    # for singular triangles           
#     for indfi, fi in enumerate(singular):
#         marked = []
#         for es in range(3):
#             ee = (es + 1) % 3
#             search = 

    # create start points and end points
    edges_start = np.zeros((len(edges[0]),3))
    edges_end = np.zeros((len(edges[1]),3))
    for inde, ei in enumerate(edges[0]):
        edges_start[inde] = ei
        edges_end[inde] = edges[1][inde]
    
    return edges_start, edges_end

start_max, end_max = extract_feature_line(v,f,kmax,kmin,kdmax,EX_max,regular, singular, 1)
start_min, end_min = extract_feature_line(v,f,kmax,kmin,kdmin,EX_min,regular, singular, -1)

# Draw Feature Lines

In [72]:
p = mp.plot(v, f, c=color)
p.add_lines(start_max, end_max, shading={"line_color": "red"})
p.add_lines(start_min, end_min, shading={"line_color": "blue"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.8809995…

2